<a href="https://colab.research.google.com/github/Cellestine/DS/blob/main/feature_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install category_encoders
import pandas as pd
import numpy as np
from category_encoders import CountEncoder

# Charger les données
test_df = pd.read_csv("/var/test_input_5qJzHrr.csv")
print("1 fait")
train_df = pd.read_csv(r"/var/train_input_Z61KlZo.csv")
print("2 fait")

# Identifier les colonnes numériques et catégorielles
num_cols = train_df.select_dtypes(include=['number']).columns.tolist()
cat_cols = train_df.select_dtypes(exclude=['number']).columns.tolist()
print("3 fait")

# REMPLACEMENT DES VALEURS MANQUANTES
print("Traitement des valeurs manquantes...")

## Remplacement des NaN numériques par 0
for col in num_cols:
    if col in train_df.columns:
        train_df[col] = train_df[col].fillna(0)
        test_df[col] = test_df[col].fillna(0)

## Remplacement des NaN catégoriques par "Inconnu"
for col in cat_cols:
    if col in train_df.columns:
        train_df[col] = train_df[col].fillna("Inconnu")
        test_df[col] = test_df[col].fillna("Inconnu")

print("Remplacement des NaN terminé.")

# ENCODAGE DES VARIABLES CATÉGORIQUES
print("Encodage des variables catégoriques...")

encoder = CountEncoder(cols=cat_cols)  # Utilisation correcte de cat_cols
train_df = encoder.fit_transform(train_df)
test_df = encoder.transform(test_df)  # Appliquer le même encodage sur test_df

print(" Encodage terminé.")


# SUPPRESSION DES COLONNES SI TROP DE 0 OU "INCONNU"
print("Suppression des colonnes avec trop de valeurs artificielles...")

## Définition du seuil : Si plus de 40% des valeurs sont "0" ou "Inconnu", on supprime la colonne
threshold = 0.4
columns_to_drop = []

for col in train_df.columns:
    missing_ratio = (train_df[col] == 0).sum() / len(train_df) if col in num_cols else (train_df[col] == "Inconnu").sum() / len(train_df)

    if missing_ratio > threshold:
        columns_to_drop.append(col)

train_df.drop(columns=columns_to_drop, inplace=True)
test_df.drop(columns=columns_to_drop, inplace=True)

print(f"Colonnes supprimées après remplacement des NaN : {columns_to_drop}")


# SUPPRESSION DES COLONNES INUTILES (Faible variance & Corrélation forte)
print("Suppression des colonnes inutiles...")

## Suppression des colonnes avec faible variance (≈ constantes)
low_variance = train_df.var()
columns_to_drop = low_variance[low_variance < 0.01].index.tolist()

train_df.drop(columns=columns_to_drop, inplace=True)
test_df.drop(columns=columns_to_drop, inplace=True)

print(f"Colonnes supprimées pour faible variance : {columns_to_drop}")

## Suppression des colonnes très corrélées (>0.95)
corr_matrix = train_df.corr().abs()
upper_tri = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
columns_to_drop = [column for column in upper_tri.columns if any(upper_tri[column] > 0.95)]

train_df.drop(columns=columns_to_drop, inplace=True)
test_df.drop(columns=columns_to_drop, inplace=True)

print(f"Colonnes supprimées pour forte corrélation : {columns_to_drop}")

# AFFICHAGE FINAL
print(f"Nettoyage terminé : {train_df.shape[1]} colonnes restantes après nettoyage")

<ipython-input-4-b765982115dd>:7: DtypeWarning: Columns (16,17,29,30,31,126,128,129,132,133,135,138,371) have mixed types. Specify dtype option on import or set low_memory=False.
  test_df = pd.read_csv("/var/test_input_5qJzHrr.csv")


1 fait


<ipython-input-4-b765982115dd>:9: DtypeWarning: Columns (16,17,29,30,31,126,128,129,132,133,135,138,371) have mixed types. Specify dtype option on import or set low_memory=False.
  train_df = pd.read_csv(r"/var/train_input_Z61KlZo.csv")


2 fait
3 fait
Traitement des valeurs manquantes...
Remplacement des NaN terminé.
Encodage des variables catégoriques...
 Encodage terminé.
Suppression des colonnes avec trop de valeurs artificielles...
Colonnes supprimées après remplacement des NaN : ['CARACT2', 'CARACT5', 'CA1', 'CA2', 'CA3', 'KAPITAL1', 'KAPITAL2', 'KAPITAL3', 'KAPITAL4', 'KAPITAL5', 'KAPITAL6', 'KAPITAL7', 'KAPITAL8', 'KAPITAL9', 'KAPITAL10', 'KAPITAL11', 'KAPITAL13', 'KAPITAL14', 'KAPITAL15', 'KAPITAL16', 'KAPITAL17', 'KAPITAL18', 'KAPITAL19', 'KAPITAL20', 'KAPITAL21', 'KAPITAL22', 'KAPITAL23', 'KAPITAL24', 'KAPITAL26', 'KAPITAL27', 'KAPITAL28', 'KAPITAL29', 'KAPITAL30', 'KAPITAL31', 'KAPITAL33', 'SURFACE5', 'SURFACE7', 'SURFACE8', 'SURFACE9', 'SURFACE12', 'SURFACE13', 'SURFACE14', 'SURFACE15', 'SURFACE16', 'SURFACE17', 'SURFACE18', 'SURFACE19', 'SURFACE20', 'SURFACE21', 'NBBAT2', 'NBBAT3', 'NBBAT5', 'NBBAT6', 'NBBAT7', 'NBBAT8', 'NBBAT9', 'NBBAT10', 'NBBAT11', 'NBBAT14', 'TAILLE3', 'TAILLE4', 'NBSINCONJ', 'NBSINST

In [ ]:
# Vérifier s'il reste des NaN après le nettoyage
print("Nombre de valeurs manquantes restantes :")
print(train_df.isnull().sum().sum())

# Vérifier si toutes les colonnes sont bien numériques
print("\nTypes de données après encodage :")
print(train_df.dtypes.value_counts())

# Vérifier la distribution des valeurs
print("\nStatistiques descriptives :")
print(train_df.describe())

pd.set_option('display.max_columns', 375)
display(test_df.head())


Nombre de valeurs manquantes restantes :
0

Types de données après encodage :
int64      118
float64      9
Name: count, dtype: int64

Statistiques descriptives :
                  ID       ACTIVIT2       VOCATION         TYPERS  \
count  383610.000000  383610.000000  383610.000000  383610.000000   
mean   191805.500000  237550.333193  172737.184583       1.665368   
std    110738.812719  109430.753732   95954.724291       0.471862   
min         1.000000     327.000000      75.000000       1.000000   
25%     95903.250000  296563.000000   67446.000000       1.000000   
50%    191805.500000  296563.000000  244441.000000       2.000000   
75%    287707.750000  296563.000000  244441.000000       2.000000   
max    383610.000000  296563.000000  244441.000000       2.000000   

          ANCIENNETE          ADOSS        CARACT1        CARACT3  \
count  383610.000000  383610.000000  383610.000000  383610.000000   
mean        5.175517  367842.108021  219210.792560  353837.834712   
std     

,ID,ACTIVIT2,VOCATION,TYPERS,ANCIENNETE,ADOSS,CARACT1,CARACT3,INDEM1,DUREE_REQANEUF,CARACT4,TYPBAT1,INDEM2,TYPBAT2,FRCH1,FRCH2,DEROG1,DEROG2,DEROG3,DEROG4,DEROG5,DEROG6,DEROG7,DEROG9,DEROG10,DEROG11,DEROG12,DEROG13,DEROG14,DEROG15,DEROG16,TAILLE1,TAILLE2,KAPITAL12,KAPITAL25,KAPITAL32,KAPITAL34,KAPITAL35,KAPITAL36,KAPITAL37,KAPITAL38,KAPITAL39,KAPITAL40,KAPITAL41,KAPITAL42,KAPITAL43,SURFACE1,SURFACE3,SURFACE4,SURFACE10,SURFACE11,NBBAT1,COEFASS,RISK1,RISK6,RISK7,RISK8,RISK9,RISK10,RISK11,RISK12,RISK13,EQUIPEMENT2,EQUIPEMENT4,EQUIPEMENT5,EQUIPEMENT6,DISTANCE_111,DISTANCE_211,DISTANCE_231,DISTANCE_242,DISTANCE_311,DISTANCE_312,DISTANCE_313,PROPORTION_11,PROPORTION_12,PROPORTION_13,PROPORTION_21,PROPORTION_22,PROPORTION_23,PROPORTION_24,PROPORTION_31,PROPORTION_32,PROPORTION_33,PROPORTION_41,PROPORTION_42,PROPORTION_51,PROPORTION_52,MEN,MEN_PAUV,MEN_1IND,MEN_5IND,MEN_PROP,MEN_FMP,MEN_COLL,MEN_MAIS,LOG_AVA1,LOG_A1_A2,LOG_A2_A3,LOG_APA3,LOG_SOC,IND_Y3_Y4,IND_Y4_Y5,IND_Y5_Y6,IND_Y7_Y8,IND_Y8_Y9,IND_Y9,IND_SNV,MEN_SURF,DISTANCE_2,ALTITUDE_2,ALTITUDE_4,ALTITUDE_5,BDTOPO_BAT_MAX_HAUTEUR_MAX,ZONE_VENT,NB_CASERNES,NBJTNS25_MM_A,NBJTNI20_MM_A,NBJFF28_MMAX_A,FXI3SAB_VOR_MM_A,FFM_VOR_COM_MM_A_Y,FFM_VOR_COM_MMAX_A_Y,FXI3SAB_VOR_COM_MM_A_Y,NBJRR100_MM_A,ANNEE_ASSURANCE,ESPINSEE,AN_EXERC,ZONE
0,383611,296563,244441,2,4,375557,93158,368284,365728,0.0,365009,353336,3404,0,102148,26573,383529,370561,375624,359142,365019,381612,382831,381410,382150,378858,349078,382317,383602,100.0,380358,79214,95799,125000,1000,87500,369780,374364,382723,365712,383442,382012,220059,319270,366640,346139,750,0.0,97717,0.0,0.0,5,340077,0.0,26573,0.0,26573,26573,87340,85903,26573,156466,87969,1,151542,5,73825,144314,133410,122376,150811,162324,161846,319870,358965,364781,33049,312169,59057,163562,17500,329035.0,361719,363473,363390,362027,362972,362312,115306,160610,350980,124689,112528,236355,227485,73293,250558,132692,114582,319281,60165,362926,334272,349457,273630,320197,148819,128509,161257,90698,105864,86817,55760,2.0,334093,116637,35755,119020,21571,125845,130869,281064,144521,0.813699,151542,26573,3
1,383612,296563,244441,2,6,375557,93158,368284,365728,0.0,365009,353336,67948,0,102148,26573,383529,370561,375624,24468,365019,381612,382831,381410,382150,378858,349078,382317,383602,100.0,380358,126652,86531,125000,4000,62500,369780,374364,382723,365712,383442,382012,220059,319270,366640,346139,175,0.0,75374,50.0,0.0,4,340077,0.0,26573,0.0,26573,26573,87340,85903,26573,156466,87969,1,151542,5,217712,217712,217712,217712,219731,218792,218907,319870,358965,364781,134467,312169,28098,163562,9976,329035.0,361719,363473,363390,362027,362972,362312,115306,147462,350980,124689,252432,236355,227485,54065,250558,132692,114582,319281,60165,362926,334272,349457,273630,320197,148819,128509,218659,33470,38514,32469,237385,1.0,334093,217712,217712,217712,217712,125845,130869,281064,217712,1.000000,151542,26573,3
2,383613,296563,38967,2,2,375557,274335,368284,365728,0.0,365009,353336,67948,0,102148,26573,383529,370561,375624,359142,365019,381612,382831,381410,382150,378858,349078,382317,383602,100.0,380358,126652,66538,125000,11000,0,369780,374364,382723,365712,383442,382012,220059,319270,366640,346139,100,0.0,75374,50.0,0.0,2,340077,0.0,26573,0.0,26573,26573,140744,140728,26573,156466,87969,1,151542,1,217712,217712,217712,217712,219731,218792,218907,319870,358965,364781,33039,312169,21040,163562,149794,329035.0,361719,363473,363390,362027,362972,362312,115306,160610,350980,124689,252432,236355,227485,73293,250558,132692,94401,319281,60165,362926,334272,349457,273630,320197,148819,128509,218659,235954,105864,240256,237385,2.0,29512,217712,217712,217712,217712,125845,130869,80715,217712,0.586301,151542,26573,3
3,383614,14359,14359,2,2,375557,274335,368284,365728,0.0,365009,353336,67948,0,102148,26573,383529,370561,375624,359142,365019,381612,382831,381410,382150,378858,349078,382317,383602,100.0,380358,50552,55253,0,3000,0,369780,374364,382723,365712,383442,382012,220059,319270,366

In [ ]:
# SAUVEGARDE DES FICHIERS NETTOYÉS EN EXCEL
train_cleaned_csv_path = "/var/train_cleaned.csv"
test_cleaned_csv_path = "/var/test_cleaned.csv"

train_df.to_csv(train_cleaned_csv_path, index=False)
test_df.to_csv(test_cleaned_csv_path, index=False)

# Afficher les chemins des fichiers sauvegardés
train_cleaned_csv_path, test_cleaned_csv_path

# SAUVEGARDE FINAL
print("save done")

save done


In [ ]:
from xgboost import XGBClassifier, XGBRegressor
from sklearn.metrics import mean_squared_error
import pandas as pd

x_train_clean = pd.read_csv(r"/var/train_cleaned.csv")
y_train = pd.read_csv(r"/var/train_output_DzPxaPY.csv")
x_test_clean = pd.read_csv(r"/var/test_cleaned.csv")

print("x_train_clean shape:", x_train_clean.shape)
print("y_train['FREQ'] shape:", y_train['FREQ'].shape)


# Correction : Assurer que y_train est bien un vecteur
boost = XGBRegressor(objective="count:poisson")

boost.fit(x_train_clean, y_train['FREQ'])

# prediction
p_freq = boost.predict(x_train_clean)
print(p_freq)

# Calculer l'erreur quadratique moyenne (MSE)
print("MSE Train -->", round(mean_squared_error(y_train['FREQ'], p_freq), 2))
print("Score Train -->", round(boost.score(x_train_clean, y_train['FREQ']) * 100, 2), "%")

# prediction
p_freq_b = boost.predict(x_test_clean)
print(p_freq_b)

from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer

# Métrique personnalisée
mse_scorer = make_scorer(mean_squared_error, greater_is_better=False)

# Cross-validation 5 folds
scores = cross_val_score(boost, x_train_clean, y_train['FREQ'], cv=5, scoring=mse_scorer)
print("Cross-validated MSE (moyenne) :", round(-np.mean(scores), 4))


x_train_clean shape: (383610, 127)
y_train['FREQ'] shape: (383610,)
[0.0095272  0.00790791 0.00765957 ... 0.01678609 0.00188686 0.00226352]
MSE Train --> 0.02
Score Train --> 81.97 %
[0.02063687 0.00563605 0.0048409  ... 0.00772263 0.01900824 0.0051262 ]
Cross-validated MSE (moyenne) : 0.1319


In [ ]:
pd.set_option('display.max_columns', 375)

p_freq_b = pd.DataFrame(p_freq_b)

display(p_freq_b.head())

x_test_clean['FREQ'] = p_freq_b

,0
0,0.020637
1,0.005636
2,0.004841
3,0.001617
4,0.009536


In [ ]:
pd.set_option('display.max_columns', 375)

# Afficher un aperçu des données pour vérifier leur chargement
print("Aperçu de Train Cleaned Data :")
display(train_df.head())
print("Aperçu de y_train :")
display(y_train.head())
print("Aperçu de test Cleaned Data :")
display(x_test_clean.head())
display(test_df.head())


Aperçu de Train Cleaned Data :


,ID,ACTIVIT2,VOCATION,TYPERS,ANCIENNETE,ADOSS,CARACT1,CARACT3,INDEM1,DUREE_REQANEUF,CARACT4,TYPBAT1,INDEM2,TYPBAT2,FRCH1,FRCH2,DEROG1,DEROG2,DEROG3,DEROG4,DEROG5,DEROG6,DEROG7,DEROG9,DEROG10,DEROG11,DEROG12,DEROG13,DEROG14,DEROG15,DEROG16,TAILLE1,TAILLE2,KAPITAL12,KAPITAL25,KAPITAL32,KAPITAL34,KAPITAL35,KAPITAL36,KAPITAL37,KAPITAL38,KAPITAL39,KAPITAL40,KAPITAL41,KAPITAL42,KAPITAL43,SURFACE1,SURFACE3,SURFACE4,SURFACE10,SURFACE11,NBBAT1,COEFASS,RISK1,RISK6,RISK7,RISK8,RISK9,RISK10,RISK11,RISK12,RISK13,EQUIPEMENT2,EQUIPEMENT4,EQUIPEMENT5,EQUIPEMENT6,DISTANCE_111,DISTANCE_211,DISTANCE_231,DISTANCE_242,DISTANCE_311,DISTANCE_312,DISTANCE_313,PROPORTION_11,PROPORTION_12,PROPORTION_13,PROPORTION_21,PROPORTION_22,PROPORTION_23,PROPORTION_24,PROPORTION_31,PROPORTION_32,PROPORTION_33,PROPORTION_41,PROPORTION_42,PROPORTION_51,PROPORTION_52,MEN,MEN_PAUV,MEN_1IND,MEN_5IND,MEN_PROP,MEN_FMP,MEN_COLL,MEN_MAIS,LOG_AVA1,LOG_A1_A2,LOG_A2_A3,LOG_APA3,LOG_SOC,IND_Y3_Y4,IND_Y4_Y5,IND_Y5_Y6,IND_Y7_Y8,IND_Y8_Y9,IND_Y9,IND_SNV,MEN_SURF,DISTANCE_2,ALTITUDE_2,ALTITUDE_4,ALTITUDE_5,BDTOPO_BAT_MAX_HAUTEUR_MAX,ZONE_VENT,NB_CASERNES,NBJTNS25_MM_A,NBJTNI20_MM_A,NBJFF28_MMAX_A,FXI3SAB_VOR_MM_A,FFM_VOR_COM_MM_A_Y,FFM_VOR_COM_MMAX_A_Y,FXI3SAB_VOR_COM_MM_A_Y,NBJRR100_MM_A,ANNEE_ASSURANCE,ESPINSEE,AN_EXERC,ZONE
0,1,296563,244441,1,0,375557,274335,368284,365728,0.0,365009,353336,215687,0,9894,26573,383529,370561,375624,359142,365019,381612,382831,381410,382150,378858,349078,382317,383602,100.0,380358,42991,32244,125000,1000,125000,369780,374364,382723,365712,383442,382012,163551,319270,366640,346139,1600,0.0,37803,50.0,0.0,9,11648,0.0,26573,0.0,26573,26573,75625,156979,26573,156466,87969,1,151542,9,217712,217712,217712,217712,219731,218792,218907,319870,358965,364781,134467,312169,188198,37000,17500,329035,361719,363473,363390,362027,362972,362312,207001,160610,350980,156440,252432,69346,69346,54065,250558,132692,114582,319281,304855,362926,334272,349457,273630,320197,148819,128509,218659,90698,105864,32469,237385,2.0,29512,217712,217712,217712,217712,125845,130869,281064,217712,1.000000,151542,26573,3
1,2,296563,244441,1,0,375557,274335,368284,365728,0.0,365009,353336,28696,0,102148,26573,383529,370561,375624,359142,365019,381612,382831,381410,382150,378858,349078,382317,383602,100.0,380358,79214,95799,125000,1000,7500,369780,374364,887,365712,383442,382012,220059,319270,366640,346139,625,0.0,97717,50.0,0.0,4,340077,0.0,26573,0.0,26573,26573,75625,156979,26573,156466,87969,1,151542,9,217712,217712,217712,217712,219731,218792,218907,319870,358965,364781,28105,312169,188198,163562,36908,329035,361719,363473,363390,2415,362972,362312,115306,18298,14015,156440,252432,236355,227485,76170,250558,132692,94401,319281,60165,362926,16785,349457,11070,320197,53263,45863,218659,90698,105864,240256,237385,2.0,334093,217712,217712,217712,217712,145426,130869,80715,217712,1.000000,151542,26573,3
2,3,296563,244441,1,2,375557,93158,368284,365728,0.0,365009,353336,28696,0,102148,26573,383529,370561,375624,359142,365019,381612,382831,381410,382150,378858,349078,382317,383602,100.0,380358,126652,86531,125000,1000,0,369780,374364,887,365712,383442,382012,163551,319270,366640,346139,225,0.0,75374,200.0,0.0,4,7708,0.0,26573,0.0,26573,26573,87340,85903,26573,156466,87969,1,151542,8,217712,217712,217712,217712,219731,218792,218907,319870,358965,364781,28105,312169,188198,163562,149794,329035,361719,363473,363390,477,362972,362312,115306,160610,14015,156440,252432,236355,227485,66230,250558,132692,14077,319281,60165,362926,334272,349457,273630,320197,148819,128509,218659,90698,105864,240256,237385,2.0,334093,217712,217712,217712,217712,125845,130869,80715,217712,0.402740,151542,26573,3
3,4,296563,244441,2,0,375557,274335,368284,365728,0.0,365009,353336,67948,0,102148,26573,383529,13049,375624,359142,365019,381612,382831,381410,382150,378858,349078,382317,383602,100.0,380358,126652,86531,125000,1000,40000,369780,374364,382723,365712,383442,382012,220059,319270,366640,346139,100,0.0,75374,50

Aperçu de y_train :


,ID,FREQ,CM,ANNEE_ASSURANCE,CHARGE
0,1,0.0,0.0,1.000000,0.0
1,2,0.0,0.0,1.000000,0.0
2,3,0.0,0.0,0.402740,0.0
3,4,0.0,0.0,0.246575,0.0
4,5,0.0,0.0,0.838356,0.0


Aperçu de test Cleaned Data :


,ID,ACTIVIT2,VOCATION,TYPERS,ANCIENNETE,ADOSS,CARACT1,CARACT3,INDEM1,DUREE_REQANEUF,CARACT4,TYPBAT1,INDEM2,TYPBAT2,FRCH1,FRCH2,DEROG1,DEROG2,DEROG3,DEROG4,DEROG5,DEROG6,DEROG7,DEROG9,DEROG10,DEROG11,DEROG12,DEROG13,DEROG14,DEROG15,DEROG16,TAILLE1,TAILLE2,KAPITAL12,KAPITAL25,KAPITAL32,KAPITAL34,KAPITAL35,KAPITAL36,KAPITAL37,KAPITAL38,KAPITAL39,KAPITAL40,KAPITAL41,KAPITAL42,KAPITAL43,SURFACE1,SURFACE3,SURFACE4,SURFACE10,SURFACE11,NBBAT1,COEFASS,RISK1,RISK6,RISK7,RISK8,RISK9,RISK10,RISK11,RISK12,RISK13,EQUIPEMENT2,EQUIPEMENT4,EQUIPEMENT5,EQUIPEMENT6,DISTANCE_111,DISTANCE_211,DISTANCE_231,DISTANCE_242,DISTANCE_311,DISTANCE_312,DISTANCE_313,PROPORTION_11,PROPORTION_12,PROPORTION_13,PROPORTION_21,PROPORTION_22,PROPORTION_23,PROPORTION_24,PROPORTION_31,PROPORTION_32,PROPORTION_33,PROPORTION_41,PROPORTION_42,PROPORTION_51,PROPORTION_52,MEN,MEN_PAUV,MEN_1IND,MEN_5IND,MEN_PROP,MEN_FMP,MEN_COLL,MEN_MAIS,LOG_AVA1,LOG_A1_A2,LOG_A2_A3,LOG_APA3,LOG_SOC,IND_Y3_Y4,IND_Y4_Y5,IND_Y5_Y6,IND_Y7_Y8,IND_Y8_Y9,IND_Y9,IND_SNV,MEN_SURF,DISTANCE_2,ALTITUDE_2,ALTITUDE_4,ALTITUDE_5,BDTOPO_BAT_MAX_HAUTEUR_MAX,ZONE_VENT,NB_CASERNES,NBJTNS25_MM_A,NBJTNI20_MM_A,NBJFF28_MMAX_A,FXI3SAB_VOR_MM_A,FFM_VOR_COM_MM_A_Y,FFM_VOR_COM_MMAX_A_Y,FXI3SAB_VOR_COM_MM_A_Y,NBJRR100_MM_A,ANNEE_ASSURANCE,ESPINSEE,AN_EXERC,ZONE,FREQ
0,383611,296563,244441,2,4,375557,93158,368284,365728,0.0,365009,353336,3404,0,102148,26573,383529,370561,375624,359142,365019,381612,382831,381410,382150,378858,349078,382317,383602,100.0,380358,79214,95799,125000,1000,87500,369780,374364,382723,365712,383442,382012,220059,319270,366640,346139,750,0.0,97717,0.0,0.0,5,340077,0.0,26573,0.0,26573,26573,87340,85903,26573,156466,87969,1,151542,5,73825,144314,133410,122376,150811,162324,161846,319870,358965,364781,33049,312169,59057,163562,17500,329035.0,361719,363473,363390,362027,362972,362312,115306,160610,350980,124689,112528,236355,227485,73293,250558,132692,114582,319281,60165,362926,334272,349457,273630,320197,148819,128509,161257,90698,105864,86817,55760,2.0,334093,116637,35755,119020,21571,125845,130869,281064,144521,0.813699,151542,26573,3,0.020637
1,383612,296563,244441,2,6,375557,93158,368284,365728,0.0,365009,353336,67948,0,102148,26573,383529,370561,375624,24468,365019,381612,382831,381410,382150,378858,349078,382317,383602,100.0,380358,126652,86531,125000,4000,62500,369780,374364,382723,365712,383442,382012,220059,319270,366640,346139,175,0.0,75374,50.0,0.0,4,340077,0.0,26573,0.0,26573,26573,87340,85903,26573,156466,87969,1,151542,5,217712,217712,217712,217712,219731,218792,218907,319870,358965,364781,134467,312169,28098,163562,9976,329035.0,361719,363473,363390,362027,362972,362312,115306,147462,350980,124689,252432,236355,227485,54065,250558,132692,114582,319281,60165,362926,334272,349457,273630,320197,148819,128509,218659,33470,38514,32469,237385,1.0,334093,217712,217712,217712,217712,125845,130869,281064,217712,1.000000,151542,26573,3,0.005636
2,383613,296563,38967,2,2,375557,274335,368284,365728,0.0,365009,353336,67948,0,102148,26573,383529,370561,375624,359142,365019,381612,382831,381410,382150,378858,349078,382317,383602,100.0,380358,126652,66538,125000,11000,0,369780,374364,382723,365712,383442,382012,220059,319270,366640,346139,100,0.0,75374,50.0,0.0,2,340077,0.0,26573,0.0,26573,26573,140744,140728,26573,156466,87969,1,151542,1,217712,217712,217712,217712,219731,218792,218907,319870,358965,364781,33039,312169,21040,163562,149794,329035.0,361719,363473,363390,362027,362972,362312,115306,160610,350980,124689,252432,236355,227485,73293,250558,132692,94401,319281,60165,362926,334272,349457,273630,320197,148819,128509,218659,235954,105864,240256,237385,2.0,29512,217712,217712,217712,217712,125845,130869,80715,217712,0.586301,151542,26573,3,0.004841
3,383614,14359,14359,2,2,375557,274335,368284,365728,0.0,365009,353336,67948,0,102148,26573,383529,370561,375624,359142,365019,381612,382831,381410,382150,378858,349078,382317,383602,100.0,380358,50552,55253,0,3000,0,369780,374364,382723,365712

,ID,ACTIVIT2,VOCATION,TYPERS,ANCIENNETE,ADOSS,CARACT1,CARACT3,INDEM1,DUREE_REQANEUF,CARACT4,TYPBAT1,INDEM2,TYPBAT2,FRCH1,FRCH2,DEROG1,DEROG2,DEROG3,DEROG4,DEROG5,DEROG6,DEROG7,DEROG9,DEROG10,DEROG11,DEROG12,DEROG13,DEROG14,DEROG15,DEROG16,TAILLE1,TAILLE2,KAPITAL12,KAPITAL25,KAPITAL32,KAPITAL34,KAPITAL35,KAPITAL36,KAPITAL37,KAPITAL38,KAPITAL39,KAPITAL40,KAPITAL41,KAPITAL42,KAPITAL43,SURFACE1,SURFACE3,SURFACE4,SURFACE10,SURFACE11,NBBAT1,COEFASS,RISK1,RISK6,RISK7,RISK8,RISK9,RISK10,RISK11,RISK12,RISK13,EQUIPEMENT2,EQUIPEMENT4,EQUIPEMENT5,EQUIPEMENT6,DISTANCE_111,DISTANCE_211,DISTANCE_231,DISTANCE_242,DISTANCE_311,DISTANCE_312,DISTANCE_313,PROPORTION_11,PROPORTION_12,PROPORTION_13,PROPORTION_21,PROPORTION_22,PROPORTION_23,PROPORTION_24,PROPORTION_31,PROPORTION_32,PROPORTION_33,PROPORTION_41,PROPORTION_42,PROPORTION_51,PROPORTION_52,MEN,MEN_PAUV,MEN_1IND,MEN_5IND,MEN_PROP,MEN_FMP,MEN_COLL,MEN_MAIS,LOG_AVA1,LOG_A1_A2,LOG_A2_A3,LOG_APA3,LOG_SOC,IND_Y3_Y4,IND_Y4_Y5,IND_Y5_Y6,IND_Y7_Y8,IND_Y8_Y9,IND_Y9,IND_SNV,MEN_SURF,DISTANCE_2,ALTITUDE_2,ALTITUDE_4,ALTITUDE_5,BDTOPO_BAT_MAX_HAUTEUR_MAX,ZONE_VENT,NB_CASERNES,NBJTNS25_MM_A,NBJTNI20_MM_A,NBJFF28_MMAX_A,FXI3SAB_VOR_MM_A,FFM_VOR_COM_MM_A_Y,FFM_VOR_COM_MMAX_A_Y,FXI3SAB_VOR_COM_MM_A_Y,NBJRR100_MM_A,ANNEE_ASSURANCE,ESPINSEE,AN_EXERC,ZONE
0,383611,296563,244441,2,4,375557,93158,368284,365728,0.0,365009,353336,3404,0,102148,26573,383529,370561,375624,359142,365019,381612,382831,381410,382150,378858,349078,382317,383602,100.0,380358,79214,95799,125000,1000,87500,369780,374364,382723,365712,383442,382012,220059,319270,366640,346139,750,0.0,97717,0.0,0.0,5,340077,0.0,26573,0.0,26573,26573,87340,85903,26573,156466,87969,1,151542,5,73825,144314,133410,122376,150811,162324,161846,319870,358965,364781,33049,312169,59057,163562,17500,329035.0,361719,363473,363390,362027,362972,362312,115306,160610,350980,124689,112528,236355,227485,73293,250558,132692,114582,319281,60165,362926,334272,349457,273630,320197,148819,128509,161257,90698,105864,86817,55760,2.0,334093,116637,35755,119020,21571,125845,130869,281064,144521,0.813699,151542,26573,3
1,383612,296563,244441,2,6,375557,93158,368284,365728,0.0,365009,353336,67948,0,102148,26573,383529,370561,375624,24468,365019,381612,382831,381410,382150,378858,349078,382317,383602,100.0,380358,126652,86531,125000,4000,62500,369780,374364,382723,365712,383442,382012,220059,319270,366640,346139,175,0.0,75374,50.0,0.0,4,340077,0.0,26573,0.0,26573,26573,87340,85903,26573,156466,87969,1,151542,5,217712,217712,217712,217712,219731,218792,218907,319870,358965,364781,134467,312169,28098,163562,9976,329035.0,361719,363473,363390,362027,362972,362312,115306,147462,350980,124689,252432,236355,227485,54065,250558,132692,114582,319281,60165,362926,334272,349457,273630,320197,148819,128509,218659,33470,38514,32469,237385,1.0,334093,217712,217712,217712,217712,125845,130869,281064,217712,1.000000,151542,26573,3
2,383613,296563,38967,2,2,375557,274335,368284,365728,0.0,365009,353336,67948,0,102148,26573,383529,370561,375624,359142,365019,381612,382831,381410,382150,378858,349078,382317,383602,100.0,380358,126652,66538,125000,11000,0,369780,374364,382723,365712,383442,382012,220059,319270,366640,346139,100,0.0,75374,50.0,0.0,2,340077,0.0,26573,0.0,26573,26573,140744,140728,26573,156466,87969,1,151542,1,217712,217712,217712,217712,219731,218792,218907,319870,358965,364781,33039,312169,21040,163562,149794,329035.0,361719,363473,363390,362027,362972,362312,115306,160610,350980,124689,252432,236355,227485,73293,250558,132692,94401,319281,60165,362926,334272,349457,273630,320197,148819,128509,218659,235954,105864,240256,237385,2.0,29512,217712,217712,217712,217712,125845,130869,80715,217712,0.586301,151542,26573,3
3,383614,14359,14359,2,2,375557,274335,368284,365728,0.0,365009,353336,67948,0,102148,26573,383529,370561,375624,359142,365019,381612,382831,381410,382150,378858,349078,382317,383602,100.0,380358,50552,55253,0,3000,0,369780,374364,382723,365712,383442,382012,220059,319270,366

In [ ]:
# SAUVEGARDE DES FICHIERS NETTOYÉS EN EXCEL
y_train_freq_csv_path = "/var/y_train_freq.csv"

y_train_output = x_test_clean[['ID','FREQ','ANNEE_ASSURANCE']]

y_train_output.to_csv(y_train_freq_csv_path, index=False)
